<a href="https://colab.research.google.com/github/Gabbers18/Python_Projects/blob/main/Examining_Crime_Rates_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Week 3 - Data Assignment**

Gabrielle Young

Questions:
- Where are different kinds of crime occurring?
- In which areas crime is growing fastest (or dropping fastest)?
- Are certain crimes more common in certain areas?


Group data by location (zip code or type of location) and examine trends within the data.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', None)
import datetime
from plotly.subplots import make_subplots

In [ ]:
nRowsRead = 1000
crimedata = pd.read_csv('/content/crimestat (1) - crimestat (1).csv', delimiter=',', nrows = nRowsRead, index_col='INC NUMBER')
crimedata.dataframeName = 'crimestat (1).csv'


# I pulled this code and the pandas packages from Kaggle. https://kaggle.com/code/cassel/module-3-basics

In [ ]:
# Preview of the data
crimedata.head(10)

,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE
INC NUMBER,,,,,,
201600000052855,11/01/2015 0:00,01/09/2016 0:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029,SINGLE FAMILY HOUSE
201600000594484,11/01/2015 0:00,NaN,RAPE,13XX E ALMERIA RD,85006,SINGLE FAMILY HOUSE
201500002102327,11/01/2015 0:00,11/01/2015 9:00,LARCENY-THEFT,51XX N 15TH ST,85014,APARTMENT
201500002101405,11/01/2015 0:00,11/01/2015 5:00,MOTOR VEHICLE THEFT,102XX W MEDLOCK AVE,85307,SINGLE FAMILY HOUSE
201500002102668,11/01/2015 0:00,11/01/2015 11:50,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043,SINGLE FAMILY HOUSE
201600000527709,11/01/2015 0:00,03/22/2016 0:36,LARCENY-THEFT,33XX W CAMELBACK RD,85017,PARKING LOT
201700001722914,11/01/2015 0:00,NaN,LARCENY-THEFT,279XX N 23RD LN,85085,SINGLE FAMILY HOUSE
201700001603695,11/01/2015 0:00,03/31/2016 0:00,RAPE,38XX W CAMELBACK RD,85019,HOSPITAL
201500002168686,11/01/2015 0:00,11/11/2015 9:30,LARCENY-THEFT,14XX E HIGHLAND AVE,85014,PARKING LOT


# **Making Sense of the Data**

**Crimes by Zip Code**

In [ ]:
# renaming columns for simplicity
crimedata.rename(columns={'UCR CRIME CATEGORY': 'category', 'PREMISE TYPE': 'TYPE'}, inplace=True)

In [ ]:
# I wanted to calucate the proportion of ALL crimes that occur within a single zip code.
zip_counts = crimedata.ZIP.value_counts()

zip_proportions = zip_counts / len(crimedata)

crimedata['zip_proportion'] = crimedata['ZIP'].map(zip_proportions)
crimedata.head(10)

,OCCURRED ON,OCCURRED TO,category,100 BLOCK ADDR,ZIP,TYPE,zip_proportion
INC NUMBER,,,,,,,
201600000052855,11/01/2015 0:00,01/09/2016 0:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029,SINGLE FAMILY HOUSE,0.031
201600000594484,11/01/2015 0:00,NaN,RAPE,13XX E ALMERIA RD,85006,SINGLE FAMILY HOUSE,0.016
201500002102327,11/01/2015 0:00,11/01/2015 9:00,LARCENY-THEFT,51XX N 15TH ST,85014,APARTMENT,0.020
201500002101405,11/01/2015 0:00,11/01/2015 5:00,MOTOR VEHICLE THEFT,102XX W MEDLOCK AVE,85307,SINGLE FAMILY HOUSE,0.003
201500002102668,11/01/2015 0:00,11/01/2015 11:50,MOTOR VEHICLE THEFT,69XX W WOOD ST,85043,SINGLE FAMILY HOUSE,0.040
201600000527709,11/01/2015 0:00,03/22/2016 0:36,LARCENY-THEFT,33XX W CAMELBACK RD,85017,PARKING LOT,0.046
201700001722914,11/01/2015 0:00,NaN,LARCENY-THEFT,279XX N 23RD LN,85085,SINGLE FAMILY HOUSE,0.010
201700001603695,11/01/2015 0:00,03/31/2016 0:00,RAPE,38XX W CAMELBACK RD,85019,HOSPITAL,0.027
201500002168686,11/01/2015 0:00,11/11/2015 9:30,LARCENY-THEFT,14XX E HIGHLAND AVE,85014,PARKING LOT,0.020


One example this output shows is that 3.1% of all crimes within the Phoenix area occur in the 85029 zip code.

In [ ]:
# Grouping by ZIP code
grouped = crimedata.groupby(["ZIP"])
type(grouped)
crimedata['ZIP'] = crimedata["ZIP"].astype(str)
crimedata['ZIP'] = crimedata["ZIP"].str.strip()

# Originally I was getting an error becasue of the ZIP code's formatting issues.

In [ ]:
# An example of a single zipcode and all crimes recorded there within this dataset.
# The 85027 zipcode represents 2.1% of all crime within the Phoenix area.
grouped.get_group('85027')

,OCCURRED ON,OCCURRED TO,category,100 BLOCK ADDR,ZIP,TYPE,zip_proportion
INC NUMBER,,,,,,,
201600000623876,11/01/2015 0:01,03/27/2016 8:00,LARCENY-THEFT,202XX N 6TH DR,85027,SINGLE FAMILY HOUSE,0.021
201500002104197,11/01/2015 6:53,NaN,BURGLARY,30XX W PARKSIDE LN,85027,SINGLE FAMILY HOUSE,0.021
201500002125373,11/01/2015 15:00,11/03/2015 18:00,MOTOR VEHICLE THEFT,10XX W TONOPAH DR,85027,SINGLE FAMILY HOUSE,0.021
201500002109584,11/01/2015 15:00,11/02/2015 11:00,LARCENY-THEFT,26XX W UNION HILLS DR,85027,HOTEL / MOTEL,0.021
201600001183969,11/01/2015 22:30,12/01/2015 4:00,LARCENY-THEFT,192XX N 20TH DR,85027,SINGLE FAMILY HOUSE,0.021
201500002110411,11/02/2015 8:00,11/02/2015 13:50,BURGLARY,212XX N 30TH AVE,85027,SINGLE FAMILY HOUSE,0.021
201500002123202,11/03/2015 11:00,11/04/2015 11:00,LARCENY-THEFT,196XX N 31ST AVE,85027,VEHICLE,0.021
201500002126001,11/03/2015 17:30,11/04/2015 7:15,LARCENY-THEFT,199XX N 23RD AVE,85027,APARTMENT,0.021
201500002123017,11/03/2015 18:00,11/04/2015 9:00,LARCENY-THEFT,21XX W PARKSIDE LN,85027,OFFICE / COMMERCIAL BUILDING,0.021


In [ ]:
crimetype_by_Zip = grouped.category.value_counts(normalize=True)
crimetype_by_Zip.head(15)

ZIP    category           
85003  LARCENY-THEFT          0.700000
       BURGLARY               0.100000
       MOTOR VEHICLE THEFT    0.100000
       AGGRAVATED ASSAULT     0.100000
85004  LARCENY-THEFT          0.727273
       DRUG OFFENSE           0.090909
       BURGLARY               0.090909
       AGGRAVATED ASSAULT     0.090909
85006  LARCENY-THEFT          0.375000
       BURGLARY               0.250000
       MOTOR VEHICLE THEFT    0.250000
       RAPE                   0.062500
       AGGRAVATED ASSAULT     0.062500
85007  LARCENY-THEFT          0.363636
       DRUG OFFENSE           0.272727
Name: proportion, dtype: float64

I have created a dataset displaying where different types of crime are occurring. Crimes are organized by zip code (location) and category (either larceny-theft, drug offense, motor vehicle theft, aggravated assault, etc.). I have also illustrated the proportion of each crime type per zip code.

According to this data, we can see that larceny-theft crimes are most common within the 85003 zip code area.

**Crimes by Location Type**

In [ ]:
# Grouping by location TYPE
locationgrouped = crimedata.groupby(["TYPE"])
type(locationgrouped)
crimedata['TYPE'] = crimedata["TYPE"].astype(str)
crimedata['TYPE'] = crimedata["TYPE"].str.strip()

In [ ]:
crimetype_by_Type = locationgrouped.category.value_counts(normalize=True)
crimetype_by_Type.head(20)

TYPE                                 category           
ADULT ONLY STORE / MOVIE             DRUG OFFENSE           1.000000
AIRPORT                              LARCENY-THEFT          0.600000
                                     DRUG OFFENSE           0.200000
                                     MOTOR VEHICLE THEFT    0.200000
APARTMENT                            MOTOR VEHICLE THEFT    0.316239
                                     LARCENY-THEFT          0.282051
                                     BURGLARY               0.247863
                                     AGGRAVATED ASSAULT     0.059829
                                     ROBBERY                0.034188
                                     RAPE                   0.034188
                                     DRUG OFFENSE           0.025641
ARENA / STADIUM / CONVENTION CENTER  LARCENY-THEFT          1.000000
BANK / SAVINGS / CREDIT UNION        LARCENY-THEFT          1.000000
BAR / LOUNG / NIGHT CLUB             AGGRAVATED ASSAULT     0.500000
                                     LARCENY-THEFT          0.500000
BUS                                  LARCENY-THEFT          1.000000
BUS STATION                          LARCENY-THEFT          1.000000
CANAL / RIGHT-OF-WAY                 DRUG OFFENSE           1.000000
CARPORT                              LARCENY-THEFT          0.666667
                                     MOTOR VEHICLE THEFT    0.333333
Name: proportion, dtype: float64

Here, I have created a dataset displaying location type where different types of crime are occurring. Crimes are organized by loation type (airport, bus station, apartment, etc.) and category (either larceny-theft, drug offense, motor vehicle theft, aggravated assault, etc.). I have also illustrated the proportion of each crime type per location type.

This displays where certain crimes are most common depending on area. For example, we can see that larceny-theft crimes are the most common type of crimes in airports.

**Crime Growth/Shrinking Rates**

In [ ]:
# Dropping NaN values to prevent errors in furture calcuations
cleancrimedata = crimedata.dropna()
cleancrimedata.head(10)
cleancrimedata['OCCURRED ON'] = pd.to_datetime(cleancrimedata['OCCURRED ON'])

<ipython-input-59-995d89ee0e5a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleancrimedata['OCCURRED ON'] = pd.to_datetime(cleancrimedata['OCCURRED ON'])
<ipython-input-59-995d89ee0e5a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleancrimedata['day'] = cleancrimedata['OCCURRED ON'].dt.day


In [ ]:
# Here, I calculated the day to day change rate for crimes by location
cleancrimedata['day'] = cleancrimedata['OCCURRED ON'].dt.day

crime_trend = cleancrimedata.groupby(['ZIP', 'day']).size().reset_index(name='crime_count')

crime_trend['crime_rate_change'] = crime_trend.groupby('ZIP')['crime_count'].pct_change()

Crime Growing Areas

In [ ]:
# Calculate crime growing areas by sorting from largest rate of change to smallest
crime_growing_areas = crime_trend.sort_values(by='crime_rate_change', ascending=False)
print(crime_growing_areas.head())

       ZIP  day  crime_count  crime_rate_change
126  85031    4            8                7.0
111  85027    5            5                4.0
179  85043    6            5                4.0
130  85032    2            5                4.0
82   85020    6            4                3.0


Here, we can see that crime rates are increasing day-to-day within the 85031 zip code.



Crime Shrinking Areas

In [ ]:
# Calculate crime shrinking areas by sorting from smallest rate of change to largest
crime_shrinking_areas = crime_trend.sort_values(by='crime_rate_change', ascending=True)
print(crime_shrinking_areas.head())

       ZIP  day  crime_count  crime_rate_change
178  85043    5            1          -0.916667
167  85041    7            1          -0.857143
171  85042    4            1          -0.833333
140  85033    7            1          -0.833333
121  85029    5            1          -0.800000


Here, we can see that crime rates are decreasing day-to-day within the 85043 zip code.
